In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1

# If required, create a face detection pipeline using MTCNN:
mtcnn = MTCNN(image_size=320, margin=0)

# Create an inception resnet (in eval mode):
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
from PIL import Image

img = Image.open("test.jpg")

# Get cropped and prewhitened image tensor
img_cropped = mtcnn(img, save_path="tensor_out.png")

# # Calculate embedding (unsqueeze to add batch dimension)
img_embedding = resnet(img_cropped.unsqueeze(0))

In [ ]:
img_embedding